In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from model_xregopose import xR_egoposemodel
from tqdm import tqdm

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=16,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = xR_egoposemodel(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		img = test_batch['image']
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {'image':img}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['pred_pose']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
			break
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	# io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp691'+'/*'))
path_

['/workspace/2d_to_3d/apps/exp691/0.pth',
 '/workspace/2d_to_3d/apps/exp691/1.pth',
 '/workspace/2d_to_3d/apps/exp691/2.pth',
 '/workspace/2d_to_3d/apps/exp691/3.pth',
 '/workspace/2d_to_3d/apps/exp691/4.pth',
 '/workspace/2d_to_3d/apps/exp691/5.pth',
 '/workspace/2d_to_3d/apps/exp691/6.pth',
 '/workspace/2d_to_3d/apps/exp691/7.pth',
 '/workspace/2d_to_3d/apps/exp691/8.pth',
 '/workspace/2d_to_3d/apps/exp691/9.pth',
 '/workspace/2d_to_3d/apps/exp691/best.pth']

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()
raise

RuntimeError: No active exception to reraise

In [5]:
for i,p in enumerate(path_):
	output_path = f'/dataset/data/xr_egopose_depth/baseline689_{i}.json'
	main(model_path=p,output_path=output_path)

07:27:20 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use Interpolat

{'FullBody': {'All': 220.48646545410156, 'UpperStretching': 221.94874572753906, 'Reacting': 210.47837829589844, 'Gesticuling': 172.9526824951172, 'LowerStretching': 232.75401306152344, 'Gaming': 215.35089111328125, 'Walking': 212.0641632080078, 'Patting': 239.73370361328125, 'Talking': 165.10923767089844, 'Greeting': 178.73841857910156}, 'UpperBody': {'All': 172.5986328125, 'UpperStretching': 185.0995635986328, 'Reacting': 163.19439697265625, 'Gesticuling': 150.34722900390625, 'LowerStretching': 145.2450714111328, 'Gaming': 147.4368133544922, 'Walking': 166.81993103027344, 'Patting': 166.6124725341797, 'Talking': 128.64332580566406, 'Greeting': 167.50344848632812}, 'LowerBody': {'All': 268.3742980957031, 'UpperStretching': 258.79791259765625, 'Reacting': 257.76239013671875, 'Gesticuling': 195.55810546875, 'LowerStretching': 320.262939453125, 'Gaming': 283.26495361328125, 'Walking': 257.30841064453125, 'Patting': 312.85491943359375, 'Talking': 201.5751495361328, 'Greeting': 189.97344970

  4%|▎         | 199/5352 [01:18<33:43,  2.55it/s]
07:28:42 [INFO] Main: Saving evaluation results...
07:28:42 [INFO] Main: Done.
07:28:42 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:

{'FullBody': {'All': 192.32716369628906, 'LowerStretching': 193.53395080566406, 'UpperStretching': 194.53091430664062, 'Walking': 199.32919311523438, 'Gesticuling': 145.09649658203125, 'Reacting': 169.78555297851562, 'Gaming': 206.04664611816406, 'Patting': 197.95423889160156, 'Greeting': 143.9148712158203, 'Talking': 155.14666748046875}, 'UpperBody': {'All': 151.67869567871094, 'LowerStretching': 122.54210662841797, 'UpperStretching': 164.6830291748047, 'Walking': 144.17530822753906, 'Gesticuling': 133.04226684570312, 'Reacting': 139.74667358398438, 'Gaming': 139.0625457763672, 'Patting': 139.93275451660156, 'Greeting': 121.53994750976562, 'Talking': 120.4312744140625}, 'LowerBody': {'All': 232.9756622314453, 'LowerStretching': 264.5257873535156, 'UpperStretching': 224.3787841796875, 'Walking': 254.48306274414062, 'Gesticuling': 157.1507568359375, 'Reacting': 199.8244171142578, 'Gaming': 273.03070068359375, 'Patting': 255.9757080078125, 'Greeting': 166.28981018066406, 'Talking': 189.8

  4%|▎         | 199/5352 [01:15<32:25,  2.65it/s]
07:30:01 [INFO] Main: Saving evaluation results...
07:30:01 [INFO] Main: Done.
07:30:01 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:

...

  4%|▎         | 199/5352 [01:15<32:29,  2.64it/s]
07:39:12 [INFO] Main: Saving evaluation results...
07:39:12 [INFO] Main: Done.
07:39:12 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:

{'FullBody': {'All': 246.2384033203125, 'UpperStretching': 243.1229705810547, 'LowerStretching': 278.41253662109375, 'Patting': 242.02171325683594, 'Reacting': 200.5012664794922, 'Gesticuling': 248.7440948486328, 'Walking': 232.57627868652344, 'Greeting': 178.9261932373047, 'Gaming': 267.2591857910156, 'Talking': 189.3408203125}, 'UpperBody': {'All': 184.35855102539062, 'UpperStretching': 195.8067169189453, 'LowerStretching': 168.01527404785156, 'Patting': 177.22889709472656, 'Reacting': 146.65379333496094, 'Gesticuling': 152.32534790039062, 'Walking': 179.0477752685547, 'Greeting': 161.82345581054688, 'Gaming': 149.26515197753906, 'Talking': 165.62588500976562}, 'LowerBody': {'All': 308.1181945800781, 'UpperStretching': 290.4392395019531, 'LowerStretching': 388.809814453125, 'Patting': 306.8145751953125, 'Reacting': 254.3487548828125, 'Gesticuling': 345.162841796875, 'Walking': 286.1047668457031, 'Greeting': 196.0289306640625, 'Gaming': 385.25323486328125, 'Talking': 213.0557250976562

  4%|▎         | 199/5352 [01:15<32:30,  2.64it/s]
07:40:31 [INFO] Main: Saving evaluation results...
07:40:31 [INFO] Main: Done.
07:40:31 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:

{'FullBody': {'All': 197.46841430664062, 'UpperStretching': 202.56642150878906, 'Patting': 202.65301513671875, 'Talking': 155.8409423828125, 'LowerStretching': 190.79058837890625, 'Reacting': 182.41812133789062, 'Walking': 204.12429809570312, 'Gaming': 193.0894012451172, 'Gesticuling': 146.67832946777344, 'Greeting': 149.52005004882812}, 'UpperBody': {'All': 153.7076416015625, 'UpperStretching': 167.67613220214844, 'Patting': 140.97698974609375, 'Talking': 121.64970397949219, 'LowerStretching': 118.05442810058594, 'Reacting': 153.656494140625, 'Walking': 144.69415283203125, 'Gaming': 146.3895263671875, 'Gesticuling': 133.46588134765625, 'Greeting': 123.96192169189453}, 'LowerBody': {'All': 241.2292022705078, 'UpperStretching': 237.4567108154297, 'Patting': 264.32904052734375, 'Talking': 190.03216552734375, 'LowerStretching': 263.5267333984375, 'Reacting': 211.17979431152344, 'Walking': 263.5544738769531, 'Gaming': 239.7892608642578, 'Gesticuling': 159.89076232910156, 'Greeting': 175.07

  4%|▎         | 199/5352 [01:15<32:25,  2.65it/s]
07:41:50 [INFO] Main: Saving evaluation results...
07:41:50 [INFO] Main: Done.
